In [1]:
#!/usr/bin/env python

from Coach import Coach
from games.tictactoe.TicTacToeGame import TicTacToeGame as Game
from games.tictactoe.mxnet.NNet import NNetWrapper as nn
from utils import *
import numpy
import GA
from random import shuffle
import math
import os

train_examples_file_name = "checkpoint.examples"

# These are the paths to where SageMaker mounts interesting things in your container.

prefix = '/opt/ml/'
channel_name='training'
model_path = os.path.join(prefix, 'model')
param_path = os.path.join(prefix, 'io/config/hyperparameters.json')
input_path = output_path = os.path.join(prefix + 'io/data', channel_name)

# Read in any hyperparameters that the user passed with the training job
if os.path.isfile(param_path):
    with open(param_path, 'r') as tc:
        args = dotdict(json.load(tc))

args = dotdict({
    'numIters': 2,
    'numEps': 20,
    'tempThreshold': 15,
    'updateThreshold': 0.6,
    'maxlenOfQueue': 200000,
    'numMCTSSims': 25,
    'arenaCompare': 4,
    'cpuct': 1,

    'checkpoint': './temp/',
    'load_model': False,
    'load_folder_file': ('/dev/models/8x100x50','best.pth.tar'),
    'numItersForTrainExamplesHistory': 7,
})

In [2]:
sol_per_pop = 6
num_parents = 3
num_generations = 20

"""
Creating the initial population.
    Generate population from scratch if there is no checkpoint,
    or load population from file.
"""
#
input_model_files = [ [model_path, file] for file in os.listdir(model_path) ]
input_model_files.sort()
game = Game()

In [3]:
from games.tictactoe.mxnet.TicTacToeNNet import TicTacToeNNet as onnet
args2 = dotdict({
    'lr': 0.001,
    'dropout': 0.3,
    'epochs': 1,
    'batch_size': 64,
    'cuda': False,
    'num_channels': 512,
})
z = onnet(game, args2)
from mxnet import nd
z.predict(nd.zeros((1,1,3,3)))

(
 [[0.28732005 0.88008714 1.0076691  0.7825768  0.31865382 1.0440212
   0.3939152  1.1920686  1.2976027  1.2683628 ]]
 <NDArray 1x10 @cpu(0)>, 
 [[-0.7818425]]
 <NDArray 1x1 @cpu(0)>)

In [4]:
new_population = []
if len(input_model_files) == 0:
    for i in range(0,sol_per_pop):
        new_population.append(nn(game))
else:
    print("Checkpoint File found. Read it.")
    sol_per_pop = len(input_model_files)
    for load_model_file in input_model_files:
        print(os.path.join(load_model_file[0], load_model_file[1]))
        nnet = nn(game)
        nnet.load_checkpoint(load_model_file[0], load_model_file[1])
        new_population.append(nnet)

In [5]:
"""
Creating Coach class used to generate Training Example
    Load Train Example History if exist
"""
master = Coach(Game(), args)
examples_file_path = os.path.join(input_path, train_examples_file_name)
if os.path.isfile(examples_file_path) :
    print(os.path.join(examples_file_path))
    master.loadTrainExamples(examples_file_path)
alpha_index = 0
"""
Start Genetic Algorithm
"""
ancestors = list(range(0,sol_per_pop))

In [6]:
from games.tictactoe.TicTacToeLogic import Board
Board(3)[0][0][0].asscalar()==0

True

In [7]:
board = game.getInitBoard()
print(board)
print(board.dtype)


[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]
<NDArray 1x1x3x3 @cpu(0)>
<class 'numpy.float32'>


In [8]:
Ps,v = z.predict(board)
valids = game.getValidMoves(board, 1)
Ps = Ps*valids  
import numpy as np
print(np.sum(Ps.asnumpy()))
from mxnet import nd
print(nd.sum(Ps))
for a in range(game.getActionSize()):
    print(a)

7.2039146

[7.2039146]
<NDArray 1 @cpu(0)>
0
1
2
3
4
5
6
7
8
9


In [9]:
def flipud(x):
    return nd.flip(data=x, axis=0)

def fliplr(x):
    return nd.flip(data=x, axis=1)

def rot90(x, k):
    k = k%4
    if k == 0:
        return x
    if k == 1:
        y = flipud(nd.transpose(x,axes=(0,1,2)))
    if k == 2:
        y = flipud(fliplr(x))
    if k == 3:
        y = nd.transpose(flipud(x),axes=(0,1,2))
    return y

pi = nd.array([0.,0.125,0.16666667,0.08333334,0.08333334,0.125,0.08333334,0.16666667,0.16666667,0.])
#pi_board = nd.random.uniform(shape=(3,3))
#pi_numpy = pi_board.asnumpy()
#k = 3
#print(pi_board)
#print("NDArray")
#rnd = rot90(pi_board,k)
#print(rnd.asnumpy())
#print(rnd.reshape((9)))
#print("Numpy")
#rnu = np.rot90(pi_numpy,k)
#print(rnu)
#print(rnu.ravel())

#nd.concat(rnd.reshape((9)), pi[-1], dim=0)

def getSymmetries(board, pi):
    # mirror, rotational
    assert(len(pi) == 3**2+1)  # 1 for pass
    pi_board = nd.reshape(pi[:-1], (1, 3, 3))
    
    l = []

    for i in range(1, 5):
        for j in [True, False]:
            newB = rot90(board, i)
            newPi = rot90(pi_board, i)
            if j:
                newB = fliplr(newB)
                newPi = fliplr(newPi)
            l += [(newB, nd.concat(newPi.reshape((9)), pi[-1], dim=0))]
    return l
    
newB = nd.random.uniform(shape=(1,3,3))
fliplr(newB)

board = nd.zeros((1,1,3,3))
sym = getSymmetries(board[0], pi)
sym[0][1].dtype

numpy.float32

In [10]:
print(board)
nd.transpose(board,axes=(0,1,2,3))


[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]
<NDArray 1x1x3x3 @cpu(0)>



[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]
<NDArray 1x1x3x3 @cpu(0)>

In [11]:
alpha_padawan = new_population[alpha_index]
train_examples = master.generate(alpha_padawan)

Generate Train Examples in 20 Eps Time: 2.474s | Total: 49.48s


In [12]:
from mxnet import nd, gpu, gluon, init, autograd
input_boards, target_pis, target_vs = list(zip(*train_examples))
dataset_train = gluon.data.dataset.ArrayDataset(input_boards, target_pis, target_vs)
data_loader = gluon.data.DataLoader(dataset_train,batch_size=64,shuffle=True,num_workers=4)

In [13]:
for input_board, target_pi, target_v in data_loader:
    print(input_board.shape)
    pi,v = z.predict(input_board)
        
    print("Input Board Dtype: ", input_board.dtype)
    print("Output Pis Dtype: ", pi.dtype)
    print("Output Vs Dtype: ", target_pi.dtype)
    print("Target Pis Dtype: ", pi.dtype)
    print("Target Vs Dtype: ", target_v.dtype)
    
    print(target_pi.shape, " = ", pi.shape)
    print(target_v.shape, " = ", v.shape)

    v_loss = z.v_loss(target_v,v) 
    pi_loss = z.pi_loss(target_pi,pi)
    loss = pi_loss + v_loss
    print(loss)
    break

(64, 1, 3, 3)
Input Board Dtype:  <class 'numpy.float32'>
Output Pis Dtype:  <class 'numpy.float32'>
Output Vs Dtype:  <class 'numpy.float32'>
Target Pis Dtype:  <class 'numpy.float32'>
Target Vs Dtype:  <class 'numpy.float32'>
(64, 10)  =  (64, 10)
(64, 1)  =  (64, 1)

[21.231863 21.052818 21.18766  21.231863 21.989727 19.496254 21.990738
 21.100172 19.574512 21.156937 21.100172 19.606977 21.052818 21.156937
 19.63634  19.702454 19.702454 19.554472 21.22323  19.593252 20.115675
 21.18766  21.019604 20.13241  19.795837 19.784962 21.383862 19.593252
 21.26675  22.033484 19.593252 19.778685 21.156937 19.56837  19.536488
 21.131683 19.96339  21.107088 19.828941 19.593252 21.181616 19.546614
 21.880856 19.656792 19.536488 19.644333 21.929937 19.826292 21.889055
 19.536488 19.594643 19.748734 21.958965 21.066362 19.744139 19.456884
 19.818253 19.723186 19.770912 21.041645 19.522575 21.990738 21.239813
 19.471699]
<NDArray 64 @cpu(0)>


In [14]:
z.pi_loss(nd.zeros((3,3)),nd.ones((3,3)))


[3.295837 3.295837 3.295837]
<NDArray 3 @cpu(0)>

In [15]:
master.train(new_population, train_examples)

------ Padawan 0: 0x120f8f7f0 ------
------ Padawan 1: 0x120fab278 ------
------ Padawan 2: 0x12626c630 ------
------ Padawan 3: 0x126218390 ------
------ Padawan 4: 0x126238358 ------
------ Padawan 5: 0x1262b40b8 ------


In [16]:
fitness = GA.cal_pop_fitness(new_population, args)

Prove your value Padawans!


MXNetError: [01:17:53] src/operator/tensor/./broadcast_reduce_op.h:461: Global reduction not supported yet

Stack trace returned 10 entries:
[bt] (0) 0   libmxnet.so                         0x0000000113a42b90 libmxnet.so + 15248
[bt] (1) 1   libmxnet.so                         0x0000000113a4293f libmxnet.so + 14655
[bt] (2) 2   libmxnet.so                         0x0000000113a42569 libmxnet.so + 13673
[bt] (3) 3   libmxnet.so                         0x0000000113dfe9d4 libmxnet.so + 3930580
[bt] (4) 4   libmxnet.so                         0x0000000114f858ed MXNDListFree + 561325
[bt] (5) 5   libmxnet.so                         0x0000000114f0c7f4 MXNDListFree + 65460
[bt] (6) 6   libmxnet.so                         0x0000000114f0eec8 MXNDListFree + 75400
[bt] (7) 7   libmxnet.so                         0x0000000114f12261 MXNDListFree + 88609
[bt] (8) 8   libmxnet.so                         0x0000000114f1217f MXNDListFree + 88383
[bt] (9) 9   libmxnet.so                         0x00000001

In [ ]:
nnet_parents, parents_weights, indices = GA.select_mating_pool(new_population, fitness, num_parents)

In [ ]:
offspring_crossover = GA.crossover(parents_weights, offspring_size=sol_per_pop-num_parents)